Daniel F Carlson
Helmholtz-Zentrum Hereon
daniel.carlson@hereon.de

This notebook demonstrates how to:
1. Generate a collection of Sentinel-2 images based on the criteria: bounding box, date range, and cloud cover
2. Visualize the RGB images and switch between images using a slider 
3. Visualize the RGB images and switch between images using arrows

Note: You must have a Google Earth Engine account. If you do not have a GEE account go to https://earthengine.google.com/

This example uses Sentinel-2 data: https://developers.google.com/earth-engine/datasets/catalog/sentinel-2

In [2]:
import ee
import geemap
from ipywidgets import interact, IntSlider

# Authenticate - you may be redirected to GEE. Follow the instructions
ee.Authenticate()
ee.Initialize()

In [3]:
def get_sentinel2_images(bbox, start_date, end_date, max_cloud_cover):
    region = ee.Geometry.Rectangle(bbox)
    date_filter = ee.Filter.date(start_date, end_date)
    cloud_filter = ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)
    combined_filter = ee.Filter.And(date_filter, cloud_filter)

    sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
                 .filterBounds(region)
                 .filter(combined_filter))

    return sentinel2

def display_rgb_image(image):
    Map = geemap.Map()
    Map.centerObject(image.geometry(), 8)
    Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1000}, 'RGB')
    Map.addLayerControl()
    Map

    return Map

In [11]:
# parameters
#bbox = [-57,62,-51,65] # bounding box coordinates: west, south, east, north
bbox = [-93,27,-87,30.4]
start_date = '2017-01-01'
end_date = '2017-03-01'
max_cloud_cover = 10

In [12]:
# Get Sentinel-2 images
sentinel2_images = get_sentinel2_images(bbox, start_date, end_date, max_cloud_cover)

# Convert image collection to a list for manual slider
image_list = sentinel2_images.toList(sentinel2_images.size())
total_images = sentinel2_images.size().getInfo()

In [13]:
# Create a function to display the map and print image ID
def update_map(image_index):
    selected_image = ee.Image(image_list.get(int(image_index)))
    Map = display_rgb_image(selected_image)
    
    # Retrieve and print image ID
    image_id = selected_image.get('system:index').getInfo()
    print("Image ID:", image_id)
    
    return Map

# Create an interactive slider
slider = IntSlider(min=0, max=sentinel2_images.size().getInfo() - 1, step=1, continuous_update=False)
interact(update_map, image_index=slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='image_index', max=137), Output(…

<function __main__.update_map(image_index)>

The slider can be a bit tricky to use. Instead of a slider, let's use arrows

In [10]:
import ipywidgets as widgets

In [19]:
def display_rgb_image_arrow(Map, image):
    Map.centerObject(image.geometry(), 8)
    Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1000}, 'RGB')
    Map.addLayerControl()

# Initial index
current_index = 0

# Create a global Map object
Map = geemap.Map()
display(Map)

# Display the first image
selected_image = ee.Image(image_list.get(current_index))
display_rgb_image_arrow(Map, selected_image)
image_id = selected_image.get('system:index').getInfo()
print("Image ID:", image_id)

# Create left and right arrow buttons
button_layout = widgets.Layout(width='50px', height='30px')
left_arrow = widgets.Button(description='←', layout=button_layout)
right_arrow = widgets.Button(description='→', layout=button_layout)

# Function to handle left arrow click
def on_left_arrow_click(b):
    global current_index
    current_index = (current_index - 1) % total_images
    update_map()

# Function to handle right arrow click
def on_right_arrow_click(b):
    global current_index
    current_index = (current_index + 1) % total_images
    update_map()

# Function to update the map
def update_map():
    Map.remove('RGB')
    selected_image = ee.Image(image_list.get(current_index))
    display_rgb_image_arrow(Map, selected_image)
    image_id = selected_image.get('system:index').getInfo()
    print("Image ID:", image_id)
    
# Attach click event handlers to the buttons
left_arrow.on_click(on_left_arrow_click)
right_arrow.on_click(on_right_arrow_click)

# Display the buttons
display(widgets.HBox([left_arrow, right_arrow]))

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Image ID: 20170101T165452_20170101T165446_T15RVM


Image ID: 20170101T165452_20170101T165446_T15RYM
Image ID: 20170105T162652_20170105T162845_T16RCQ
Image ID: 20170105T162652_20170105T162845_T16RCR
Image ID: 20170105T162652_20170105T162845_T16RCS
Image ID: 20170105T162652_20170105T162845_T16RDQ
Image ID: 20170105T162652_20170105T162845_T16RDR
Image ID: 20170105T162652_20170105T162845_T16RDS
Image ID: 20170105T162652_20170105T162845_T16REQ
Image ID: 20170105T162652_20170105T162845_T16RER
Image ID: 20170105T162652_20170105T162845_T16RES
Image ID: 20170108T163641_20170108T163846_T15RYN
Image ID: 20170108T163641_20170108T163846_T15RYP
Image ID: 20170108T163641_20170108T163846_T16RBT
